## Data Ingestion Sales  Data Set
We are going to load the Sales Data Set from a private Azure Storage Account.

<font size="2" color="red" face="sans-serif" bold> 

<b> <i> <u>No changes are required to this cell, This cell have all the necessary credentials to Ingest data from storage account
</font>

In [ ]:
# Providing the details for the Azure Storage account
# Mention about the SAS key
storage_account = "fabricvbdadlspublic"
container = "labdata"

# Since the container is set to private access, we need the SAS Key
# Ask the instructor for the SAS Key. This is an example of how the SAS key should look like: 

sas_token = "insert sas token" 

# Set Spark config to access  blob storage
spark.conf.set("fs.azure.sas.%s.%s.blob.core.windows.net" % (container, storage_account),sas_token)

# We specify the path for the csv files we need to read. By reading the *.csv, we are reading all the files from the folder Facts
wasbs_path1 = f"wasbs://{container}@{storage_account}.blob.core.windows.net/CSV/Facts/Sales_File1.csv"
wasbs_path2 = f"wasbs://{container}@{storage_account}.blob.core.windows.net/CSV/Facts/Sales_File2.csv"

# We are specifying the file type, csv and displaying the headers of the sales files
sales_df1 = spark.read.format("csv").option("delimiter", ";").option("header","true").option("inferSchema", "true").load(wasbs_path1)
sales_df2 = spark.read.format("csv").option("delimiter", ";").option("header","true").option("inferSchema", "true").load(wasbs_path2)

In [ ]:
# We are currently showcasing both data frames
sales_df2.show()
sales_df1.show()

In [ ]:
# We print only the schema to better understand the dataframes data types structure

sales_df1.printSchema()
sales_df1.count()


In [ ]:
# We print only the schema to better understand the dataframes data types structure. 
# As you can see these two dataframes have different columns. Hence a merge join is required.

sales_df2.printSchema()
sales_df2.count()

In [ ]:
# By ordering the data frames based on the columns that will be used for the merge join, we can significantly improve the efficiency of the join operation.
# In this case the column SalesOrderNumber
from pyspark.sql.functions import col

sorted_sales_df1 = sales_df1.orderBy(col("SalesOrderNumber"),col("SalesOrderLineNumber"))

sorted_sales_df1.show()

In [ ]:
# By ordering the data frames based on the columns that will be used for the merge join, we can significantly improve the efficiency of the join operation.
# In this case the column SalesOrderNumber
from pyspark.sql.functions import col

sorted_sales_df2 = sales_df2.orderBy(col("SalesOrderNumber"),col("SalesOrderLineNumber"))
sorted_sales_df2.show()

In [ ]:
# This cell contains an inner join operation on the 'SalesOrderNumber' and 'SalesOrderLineNumber' columns.

merged_df = sorted_sales_df1.join(
    sorted_sales_df2,
    on=["SalesOrderNumber", "SalesOrderLineNumber"],
    how="inner"
)

# Show the result
merged_df.printSchema()

##### Prior to providing a table name, we verify that no other table shares the same name.

In [ ]:
%%sql
DROP TABLE IF EXISTS Sales;


##  Sales DataSet ingestion onto a Managed Lakehouse Table


<font size="2" color="red" face="sans-serif" bold> 

<b> <i> <u>
</font>

In [ ]:
# Writing the Data Frame directly into the Delta Table from Managed Zone
table_name = 'Sales'

merged_df \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .save("Tables/" + table_name)

In [ ]:
# Now that the table has been created, we can utilize Pyspark SQL to generate a new data frame and load the table
table_name = "Sales"

# Read the table into a DataFrame
df = spark.read.table(table_name)

# Calculate the number of rows
row_count = df.count()

# Print or use the row count as needed
print("Number of rows:", row_count)

In [ ]:
# To display the content of the table as dataframe we run the display command:
display(df)